In [58]:
import mne
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal
import pandas as pd

def convert_csv(filename,sample_number=256,sensor_position='sensor position',sensor_value='sensor value',Fs=200):
    raw=pd.read_csv(filename)
    channel=raw[sensor_position].unique()
    data = np.empty((0,sample_number))
    for i in channel:
        mask=raw[sensor_position]==i
        data=np.append(data,np.array([raw[mask][sensor_value]]),axis=0)
    info=mne.create_info(ch_names=list(channel),sfreq=Fs)
    R_raw=mne.io.RawArray(data,info)
    return R_raw

def integral(a,b):
    mask=((f>a) & (f<b))
    s_new=s[0][mask]
    return s_new.sum()/s_new.shape[0]

final_input=pd.DataFrame()

for i in range(1,481):
    raw_csv=pd.read_csv('D:\Dataset\EEG\Data'+str(i)+'.csv')
    eeg_channel=raw_csv['sensor position'].unique()
    raw=convert_csv('D:\Dataset\EEG\Data'+str(i)+'.csv')
    
    psd_result=pd.DataFrame()
    for i in eeg_channel:
        x=raw.get_data(i)
        f,s=signal.periodogram(x,fs=256,scaling='density')
        new_data={
            'delta': [integral(0.2,3.99)],
            'theta' : [integral(4,7.99)],
            'alpha' : [integral(8,12.99)],
            'beta' : [integral(13,30)],
            'gamma' : [integral(30,50)]    }
        new_df=pd.DataFrame(new_data,index=[i])
        psd_result=pd.concat([psd_result,new_df])
    '''psd_result['Alcoholic']=1 if raw_csv['subject identifier'][0]=='a' else 0'''
    print(psd_result)
    input_df=pd.DataFrame(psd_result.values.reshape(-1,))
    input_df=input_df.transpose()
    input_df['Alcoholic']=1 if raw_csv['subject identifier'][0]=='a' else 0
    final_input=pd.concat([final_input,input_df])
print(final_input)

Creating RawArray with float64 data, n_channels=64, n_times=256
    Range : 0 ... 255 =      0.000 ...     1.275 secs
Ready.
        delta     theta     alpha      beta     gamma
FP1  5.740591  0.383755  0.718159  0.154499  0.132248
FP2  6.918583  0.681478  0.739321  0.517672  0.338378
F7   3.580122  0.084051  0.798157  0.492270  0.426433
F8   4.093898  0.998916  0.570787  1.974057  1.709517
AF1  3.868267  0.358736  0.486813  0.122105  0.070375
..        ...       ...       ...       ...       ...
P2   1.850268  0.417348  0.138046  0.091271  0.022177
P1   1.280101  0.452453  0.116149  0.112841  0.026140
CPZ  0.256527  0.055353  0.020251  0.019207  0.004004
nd   5.402125  0.364908  0.718468  0.149897  0.128454
Y    4.766057  0.321232  1.095223  0.422924  0.197981

[64 rows x 5 columns]
Creating RawArray with float64 data, n_channels=64, n_times=256
    Range : 0 ... 255 =      0.000 ...     1.275 secs
Ready.
        delta     theta     alpha      beta     gamma
FP1  1.151810  0.653528  

In [59]:
final_input

,0,1,2,3,4,5,6,7,8,9,...,311,312,313,314,315,316,317,318,319,Alcoholic
0,5.740591,0.383755,0.718159,0.154499,0.132248,6.918583,0.681478,0.739321,0.517672,0.338378,...,0.364908,0.718468,0.149897,0.128454,4.766057,0.321232,1.095223,0.422924,0.197981,1
0,1.151810,0.653528,0.286628,0.314008,0.140931,1.064151,0.714885,0.221922,1.304798,0.365563,...,0.635193,0.271345,0.318013,0.140613,4.142762,0.863275,0.254717,0.549159,0.249575,1
0,56.433751,4.341673,0.943086,0.620207,0.154746,45.916273,4.102767,1.015726,1.290302,0.369442,...,4.185667,0.911422,0.605171,0.153115,8.468380,0.391832,0.442159,0.632763,0.181498,1
0,55.910577,1.521050,0.430987,0.277966,0.134890,51.519215,1.717259,0.556534,0.848873,0.377550,...,1.447024,0.434253,0.277927,0.125537,9.425776,1.325465,0.355167,0.346198,0.154956,1
0,5.229340,0.386402,0.335361,0.222757,0.082150,4.260367,0.388752,0.463019,0.977324,0.285674,...,0.374776,0.318907,0.220976,0.089770,3.161741,0.964762,0.475613,0.242501,0.115237,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,30.598598,3.046381,4.723261,0.377478,0.037271,34.587356,3.742678,4.106170,0.397491,0.030535,...,5.943676,9.534309,0.488390,0.084343,39.550808,7.908414,5.793416,0.442806,0.299148,0
0,79.014465,4.076040,2.794986,0.424629,0.105623,70.064915,3.228677,2.794856,0.399844,0.099062,...,5.106547,5.964648,0.600424,0.072361,66.156590,2.658429,4.641405,0.670964,0.256599,0
0,18.191630,7.055885,6.352790,0.279375,0.045743,22.365875,5.542468,5.955831,0.289193,0.046508,...,7.552783,3.683212,0.741402,0.072799,39.639657,7.321640,9.491940,0.512336,0.133157,0
0,35.054098,1.206753,7.221822,0.581854,0.087805,27.724696,0.983737,6.255265,0.548829,0.084516,...,2.471136,2.056305,0.862750,0.119183,32.770783,4.952705,14.257915,0.866427,0.236249,0


In [93]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix


X_train,X_test,y_train,y_test=train_test_split(final_input[list(range(0,320))],final_input['Alcoholic'],test_size=0.3,random_state=1)

rf=RandomForestClassifier(n_estimators=100,random_state=1)
rf.fit(X_train,y_train)
predicted=rf.predict(X_test)
accuracy=accuracy_score(y_test,predicted)
print(accuracy)

0.9513888888888888


In [94]:
confusion_matrix(y_test, predicted)

array([[77,  2],
       [ 5, 60]], dtype=int64)